# Heuristic 평가
Heuristic 평가는 불충분한 시간이나 정보로 인해 완벽하게 합리적인 판단을 할 수 없을 때, 빠르고 간편하게 사용할 수 있는 추론 방법입니다.

(이는 LLM as Judge 를 활용할 때 드는 시간과 비용을 절약할 수 있다는 강점을 가지고 있기도 합니다.)

(참고) 아래의 코드 주석을 해제하여 라이브러리를 업데이트 후 진행합니다.

https://github.com/teddylee777/langchain-kr/blob/main/16-Evaluations/08-LangSmith-Heuristic-Evaluation.ipynb

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


### RAG 성능 테스트를 위한 함수 정의

In [3]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

rag = PDFRAG(
    "data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatOpenAI(model="gpt-4o-mini", temperature=0)
)

retriever = rag.create_retriever()

chain = rag.create_chain(retriever)

chain.invoke("삼성전자가 자체 개발산 생성형 AI의 이름은 뭐야?")

"삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."

In [4]:
def ask_question(inputs:dict):
    return {"answer": chain.invoke(inputs["question"])}

### 한글 형태소 분석기의 활용
한글 형태소 분석기는 한국어 문장을 가장 작은 의미 단위인 형태소로 분리하고 각 형태소의 품사를 판별하는 도구입니다.

형태소 분석기의 주요 기능

- 문장을 형태소 단위로 분리
- 각 형태소의 품사 태깅
- 형태소의 기본형 추출
Kiwipiepy 라이브러리를 활용하여 한글 형태소 분석기를 사용할 수 있습니다.

In [5]:
from langchain_teddynote.community.kiwi_tokenizer import KiwiTokenizer

kiwi_tokenizer = KiwiTokenizer()

sent1 = "안녕하세요. 반갑습니다. 내 이름은 재호입니다."
sent2 = "안녕하세용~ 반갑습니다^^ 내 이름은 재호입니다!!"

# 토큰화
print(sent1.split())
print(sent2.split())

print("===" * 20)

print(kiwi_tokenizer.tokenize(sent1))
print(kiwi_tokenizer.tokenize(sent2))

['안녕하세요.', '반갑습니다.', '내', '이름은', '재호입니다.']
['안녕하세용~', '반갑습니다^^', '내', '이름은', '재호입니다!!']
['안녕', '하', '세요', '.', '반갑', '습니다', '.', '나', '의', '이름', '은', '재호', '이', 'ᆸ니다', '.']
['안녕', '하', '세요', 'ᆼ', '~', '반갑습니다', '^^', '나', '의', '이름', '은', '재호', '이', 'ᆸ니다', '!!']


SEMSCORE vs 다른 평가 지표

- BLEU, ROUGE와 달리 단순한 n-gram 매칭에 의존하지 않습니다.
- METEOR보다 더 고급화된 의미적 유사성을 측정합니다.
- BERTScore와 유사하지만, 지시사항 기반 작업에 특화되어 있습니다.
`SentenceTransformer` 모델을 사용하여 문장 임베딩을 생성하고, 두 문장 간의 코사인 유사도를 계산합니다.

- 논문에서 사용된 모델인 all-mpnet-base-v2 를 사용합니다.

In [6]:
# import nltk
# nltk.download('wordnet')

In [7]:
from langsmith.schemas import Run, Example
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate import meteor_score
from sentence_transformers import SentenceTransformer, util
import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"

c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
def rouge_evaluator(metric: str = "rouge1") -> dict:
    # wrapper function 정의
    def _rouge_evaluator(run: Run, example: Example) -> dict:
        # 출력값과 정답 가져오기
        student_answer = run.outputs.get("answer", "")
        reference_answer = example.outputs.get("answer", "")

        # ROUGE 점수 계산
        scorer = rouge_scorer.RougeScorer(
            ["rouge1", "rouge2", "rougeL"], use_stemmer=True, tokenizer=KiwiTokenizer()
        )
        scores = scorer.score(reference_answer, student_answer)

        # ROUGE 점수 반환
        rouge = scores[metric].fmeasure

        return {"key": "ROUGE", "score": rouge}

    return _rouge_evaluator

In [9]:
def bleu_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # 토큰화
    reference_tokens = kiwi_tokenizer.tokenize(reference_answer, type="sentence")
    student_tokens = kiwi_tokenizer.tokenize(student_answer, type="sentence")

    # BLEU 점수 계산
    bleu_score = sentence_bleu([reference_tokens], student_tokens)

    return {"key": "BLEU", "score": bleu_score}

In [10]:
def meteor_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # 토큰화
    reference_tokens = kiwi_tokenizer.tokenize(reference_answer, type="list")
    student_tokens = kiwi_tokenizer.tokenize(student_answer, type="list")

    # METEOR 점수 계산
    meteor = meteor_score.meteor_score([reference_tokens], student_tokens)

    return {"key": "METEOR", "score": meteor}

In [11]:
def semscore_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # SentenceTransformer 모델 로드
    model = SentenceTransformer("all-mpnet-base-v2")

    # 문장 임베딩 생성
    student_embedding = model.encode(student_answer, convert_to_tensor=True)
    reference_embedding = model.encode(reference_answer, convert_to_tensor=True)

    # 코사인 유사도 계산
    cosine_similarity = util.pytorch_cos_sim(
        student_embedding, reference_embedding
    ).item()

    return {"key": "sem_score", "score": cosine_similarity}

In [12]:
from langsmith.evaluation import evaluate

# 평가자 정의
heuristic_evalulators = [
    rouge_evaluator(metric="rougeL"),
    bleu_evaluator,
    meteor_evaluator,
    semscore_evaluator,
]

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET_teddynote"

# 실험 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=heuristic_evalulators,
    experiment_prefix="Heuristic-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Heuristic-EVAL (Rouge, BLEU, METEOR, SemScore) 을 사용하여 평가",
    },
)

View the evaluation results for experiment: 'Heuristic-EVAL-4664d59f' at:
https://smith.langchain.com/o/2d0ce887-3f7f-59af-8d5e-12c1371ef5d5/datasets/334d943e-3086-4d03-91ae-4b5145224ffc/compare?selectedSessions=4ec63538-9820-4ac6-b151-b3648ab611a7




0it [00:00, ?it/s]

Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running evaluator <DynamicRunEvaluator meteor_evaluator> on run dea5fc48-055f-46d2-94bb-ac6a0adc3461: AttributeError("'WordNetCorpusReader' object has no attribute '_LazyCorpusLoader__args'")
Traceback (most recent call last):
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\langsmith\evaluation\_runner.py", line 1345, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\langsmith\evaluation\evaluator.py", line 327, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-pac